In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
raw_df_match_details = pd.read_csv('../data/raw/match_details.csv')
raw_df_player_attr = pd.read_csv('../data/raw/player_attributes.csv')

In [3]:
raw_df_player_attr

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,183974,102359,39902,2009-08-30 00:00:00,83.0,85.0,right,medium,low,84.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,183975,102359,39902,2009-02-22 00:00:00,78.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,183976,102359,39902,2008-08-30 00:00:00,77.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30 00:00:00,78.0,81.0,right,medium,low,74.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [4]:
# raw_df_match_details.info()

In [5]:
# raw_df_player_attr.info()

In [6]:
raw_df_match_details = (raw_df_match_details
                        .assign(date=pd.to_datetime(raw_df_match_details['date']),
                                result_match=lambda df: (df['result_match'] == 'H').astype(int))
                        )

In [7]:
raw_df_match_details['date'] = pd.to_datetime(raw_df_match_details['date'])
raw_df_player_attr['date'] = pd.to_datetime(raw_df_player_attr['date'])
raw_df_match_details.sort_values(by=['season', 'date'], inplace=True)
# raw_df_match_details.drop('stage', axis=1, inplace=True)

In [8]:
from playerstats import player_stats

players_cols = ['{}_player_{}'.format(team, i) for team in ['home', 'away'] for i in range(1, 12)]

player_stats_dict_series = raw_df_match_details.apply(
    lambda row: player_stats.get_player_stat(
        match_row=row,
        df_matches=raw_df_match_details,
        df_player_attr=raw_df_player_attr,
        players=players_cols
    ),
    axis=1
)

new_player_stats_df = pd.json_normalize(player_stats_dict_series)

df = pd.merge(raw_df_match_details, new_player_stats_df, how='left', on='match_api_id')
df.drop(players_cols, axis=1, inplace=True)
matching_columns = [col for col in df.columns if 'player_rating' in col] + [col for col in df.columns if
                                                                            'aggression_rating' in col] + [col for col
                                                                                                           in df.columns
                                                                                                           if
                                                                                                           'strength_rating' in col] + [
                       col for col in df.columns if 'acceleration_rating' in col]
df.dropna(subset=matching_columns, inplace=True)
matching_columns_filtered = [c for c in df.columns if c != "date" and c != "season"]

In [9]:
from src.shiftdata.shift_data import ShiftDataPreprocessor

preprocessor = ShiftDataPreprocessor(df)

home_df = preprocessor.select_and_rename_columns('home_')
away_df = preprocessor.select_and_rename_columns('away_')

team_df = preprocessor.concatenate_teams(home_df, away_df)

features_to_shift = ['team_goal', 'team_shoton', 'team_possession']
team_df_shifted = preprocessor.shift_features(features_to_shift)

df_final = preprocessor.merge_shifted_features(team_df_shifted)

INFO: Merging shifted features back into the original DataFrame.


In [54]:
df_ = df_final.copy()

In [55]:
df_[:5].to_csv('../data/preprocessed/match_details_5.csv', index=False)

In [56]:
df_

,match_api_id,season,stage,date,away_team,home_team,result_match,overall_rating_home_player_1,acceleration_rating_home_player_1,strength_rating_home_player_1,...,overall_rating_away_player_11,acceleration_rating_away_player_11,strength_rating_away_player_11,aggression_rating_away_player_11,home_prev_team_goal_shifted,home_prev_team_shoton_shifted,home_prev_team_possession_shifted,away_prev_team_goal_shifted,away_prev_team_shoton_shifted,away_prev_team_possession_shifted
18,489063,2008/2009,3,2008-08-30,10261,9825,1,78.500000,78.500000,78.500000,...,76.500000,76.500000,76.500000,76.500000,1.0,12.0,34.0,1.0,1.0,55.0
21,489068,2008/2009,3,2008-08-30,10194,8549,1,73.500000,73.500000,73.500000,...,62.000000,62.000000,62.000000,62.000000,1.0,7.0,47.0,3.0,1.0,47.0
22,489069,2008/2009,3,2008-08-30,8659,8559,0,68.000000,68.000000,68.000000,...,68.000000,68.000000,68.000000,68.000000,3.0,5.0,53.0,1.0,2.0,66.0
23,489070,2008/2009,3,2008-08-30,8528,8667,0,65.500000,65.500000,65.500000,...,71.000000,71.000000,71.000000,71.000000,1.0,5.0,47.0,1.0,7.0,52.0
25,489066,2008/2009,3,2008-08-31,8650,10252,0,78.666667,78.666667,78.666667,...,87.333333,87.333333,87.333333,87.333333,2.0,5.0,48.0,2.0,11.0,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033,1987603,2015/2016,38,2016-05-15,8654,10194,1,74.600000,74.600000,74.600000,...,77.400000,77.400000,77.400000,77.400000,1.0,4.0,47.0,3.0,12.0,52.0
3034,1987604,2015/2016,38,2016-05-15,8456,10003,0,75.400000,75.400000,75.400000,...,87.300000,87.300000,87.300000,87.300000,4.0,9.0,53.0,2.0,4.0,43.0
3035,1987605,2015/2016,38,2016-05-15,8472,9817,0,72.400000,72.400000,72.400000,...,77.700000,77.700000,77.700000,77.700000,2.0,3.0,34.0,3.0,7.0,31.0
3036,1987606,2015/2016,38,2016-05-15,8650,8659,0,77.800000,77.800000,77.800000,...,68.700000,68.700000,68.700000,68.700000,1.0,7.0,54.0,1.0,9.0,66.0


In [57]:
home_strength_rating_cols = df_.filter(like='strength_rating_home').columns.to_list()
away_strength_rating_cols = df_.filter(like='strength_rating_away').columns.to_list()

home_aggression_rating_cols = df_.filter(like='aggression_rating_home').columns.to_list()
away_aggression_rating_cols = df_.filter(like='aggression_rating_away').columns.to_list()

home_overall_rating_cols = df_.filter(like='overall_rating_home').columns.to_list()
away_overall_rating_cols = df_.filter(like='overall_rating_away').columns.to_list()

home_acceleration_rating_cols = df_.filter(like='acceleration_rating_home').columns.to_list()
away_acceleration_rating_cols = df_.filter(like='acceleration_rating_away').columns.to_list()

home_possession_rating_cols = df_.filter(like='home_prev_team_possession_shifted').columns.to_list()
away_possession_rating_cols = df_.filter(like='away_prev_team_possession_shifted').columns.to_list()

home_shoton_rating_cols = df_.filter(like='home_prev_team_shoton_shifted').columns.to_list()
away_shoton_rating_cols = df_.filter(like='away_prev_team_shoton_shifted').columns.to_list()

In [58]:
df_['team_strength_home'] = df_[home_strength_rating_cols].mean(axis=1)
df['team_strength_away'] = df_[away_strength_rating_cols].mean(axis=1)
df_['strength_difference'] = df_['team_strength_home'] - df['team_strength_away']

df_['team_aggression_home'] = df_[home_aggression_rating_cols].mean(axis=1)
df_['team_aggression_away'] = df_[home_aggression_rating_cols].mean(axis=1)

df_['team_acceleration_home'] = df_[home_acceleration_rating_cols].mean(axis=1)
df_['team_acceleration_away'] = df_[home_acceleration_rating_cols].mean(axis=1)

df_['rolling_avg_goals_home'] = df_['home_prev_team_goal_shifted'].rolling(window=5).mean()
df_['rolling_avg_goals_away'] = df_['away_prev_team_goal_shifted'].rolling(window=5).mean()

df_['goal_conversion_rate_home'] = df_['home_prev_team_goal_shifted'] / (df_['home_prev_team_shoton_shifted'] + 1)
df_['goal_conversion_rate_away'] = df_['away_prev_team_goal_shifted'] / (df_['away_prev_team_shoton_shifted'] + 1)

columns_to_drop = (
    home_strength_rating_cols
    + away_strength_rating_cols
    + home_aggression_rating_cols
    + away_aggression_rating_cols
    + home_overall_rating_cols
    + away_overall_rating_cols
    + home_possession_rating_cols
    + away_possession_rating_cols
    + home_shoton_rating_cols
    + away_shoton_rating_cols
)

# Drop the valid columns
df_ = df_.drop(columns_to_drop, axis=1)

In [60]:
df_.to_csv('../data/preprocessed/preprocessed_1.csv', index=False)